In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt



In [4]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D,Flatten,Dense

In [ ]:
import os
import shutil

def organize_images(base_dir):
    cats_dir = os.path.join(base_dir, "cats")
    dogs_dir = os.path.join(base_dir, "dogs")

    # Create folders if they don't exist
    os.makedirs(cats_dir, exist_ok=True)
    os.makedirs(dogs_dir, exist_ok=True)

    # Move images only (ignore directories)
    for filename in os.listdir(base_dir):
        file_path = os.path.join(base_dir, filename)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Move images based on filename
        if filename.lower().startswith("cat"):
            shutil.move(file_path, os.path.join(cats_dir, filename))
        elif filename.lower().startswith("dog"):
            shutil.move(file_path, os.path.join(dogs_dir, filename))

# Fix train and test directories
organize_images("E:\\archive\\train\\train")
organize_images("E:\\archive\\test\\test1")

print("Images sorted into 'cats' and 'dogs' folders.")


Images sorted into 'cats' and 'dogs' folders.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for test data

train_dir = "E:\\archive\\train\\train"
test_dir ="E:\\archive\\test\\test1"

train_generator = train_datagen.flow_from_directory(
    "E:\\archive\\train\\train",
    target_size=(150, 150),  # Resize images to match input size
    batch_size=32,
    class_mode='binary'  # Since it’s a binary classification (cats vs. dogs)
)

test_generator = test_datagen.flow_from_directory(
    "E:\\archive\\test\\test1",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

print("Train dataset size:", len(train_generator))
print("Test dataset size:", len(test_generator))


Found 25000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Train dataset size: 782
Test dataset size: 157


In [16]:
import os
print("Train directory contents:", os.listdir(train_dir)[:10])

Train directory contents: ['cats', 'dogs']


In [17]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
batch = next(iter(train_generator))  # Get one batch
print("Batch shape (images):", batch[0].shape)  # Image data
print("Batch shape (labels):", batch[1].shape)  # Labels


Batch shape (images): (32, 150, 150, 3)
Batch shape (labels): (32,)


In [19]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),  # 3 for RGB
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (cats vs dogs)
])


In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [21]:
print("Test set size:", test_generator.samples)


Test set size: 5000


In [22]:
history = model.fit(train_generator, validation_data=test_generator, epochs=10)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1793s 2s/step - accuracy: 0.5958 - loss: 1.6153 - val_accuracy: 0.6822 - val_loss: 0.5998
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2202s 3s/step - accuracy: 0.6578 - loss: 0.6217 - val_accuracy: 0.7006 - val_loss: 0.5704
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2007s 3s/step - accuracy: 0.6662 - loss: 0.6072 - val_accuracy: 0.7318 - val_loss: 0.5328
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1537s 2s/step - accuracy: 0.6808 - loss: 0.5896 - val_accuracy: 0.7300 - val_loss: 0.5293
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2384s 3s/step - accuracy: 0.6954 - loss: 0.5787 - val_accuracy: 0.7354 - val_loss: 0.5210
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1387s 2s/step - accuracy: 0.6980 - loss: 0.5746 - val_accuracy: 0.7424 - val_loss: 0.5216
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6309s 8s/step - accuracy: 0.7035 - loss: 0.5703 - val_accuracy: 0.7556 - val_loss: 0.5030
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4085s 5s/step - accuracy: 0.7122 - loss: 0.5602 - 

In [23]:
model.save("cats_vs_dogs.h5")

In [24]:
model.save("cats_vs_dogs.keras")